In [1]:
# MDP를 모를 때 최고의 정책을 찾는 방법

import random
import numpy as np


class GridWorld():
    def __init__(self): # 책에서 시작 지점을 (2, 0)으로 표기하였으나, 실제로는 (0, 0)에서 시작
        self.x = 0
        self.y = 0

    def step(self, a):
        if a == 0:
            self.move_left()
        elif a == 1:
            self.move_up()
        elif a == 2:
            self.move_right()
        elif a == 3:
            self.move_down()

        reward = -1
        done = self.is_done()
        return (self.x, self.y), reward, done # 에이전트로부터 action을 받아서 다음 상태와 보상, 에피소드가 끝났는지 여부를 리턴
    
    def move_left(self):
        if self.y == 0: # x = 0에서는 left를 해도 제자리 유지
            pass 
        elif self.y == 3 and self.x in [0, 1, 2]: # x = 3, y = 0, 1, 2에서는 left를 해도 제자리 유지 (벽이 있기 때문에)
            pass 
        elif self.y == 5 and self.x in [2, 3, 4]: # x = 5, y = 2, 3, 4에서는 left를 해도 제자리 유지 (벽이 있기 때문에)
            pass
        else:
            self.y -= 1 # 위의 경우가 아니면 left를 하면 x좌표가 1 감소

    def move_right(self):
        if self.y == 1 and self.x in [0, 1, 2]:
            pass
        elif self.y == 3 and self.x in [2, 3, 4]:
            pass
        elif self.y == 6:
            pass
        else:
            self.y += 1

    def move_up(self):
        if self.x == 0:
            pass
        elif self.y == 2 and self.x == 3:
            pass
        else:
            self.x -= 1 # 기존대로면 += 이 맞는데, 문제 정의가 위로 갈수록 index가 작아지는 구조이기 때문에 -=로 구현

    def move_down(self):
        if self.x == 4:
            pass
        elif self.y == 4 and self.x == 1:
            pass
        else:
            self.x += 1

    def is_done(self):
        if self.y == 6 and self.x == 4: # Terminal State
            return True
        else:
            return False
        
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

In [31]:
class MCAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) # 7 x 5 x 4 크기의 0으로 초기화된 Q 테이블 생성 -> why? -> action-value function을 표현하기 위해(4개의 액션에 대한 q값을 모두 저장하기 위해)
        self.eps = 0.9 # 탐색을 위한 입실론 값 설정 (eps-greedy)
        self.alpha = 0.01
        self.gamma = 1

    def select_action(self, s):
        # eps-greedy로 액션을 선택
        x, y = s
        coin = random.random() # 0 ~ 1 사이의 값
        if coin < self.eps:
            action = random.randint(0, 3) # 0, 1, 2, 3 중 하나를 랜덤하게 선택 (90% 확률로 exploration -> 0.1까지 decaying)
        else:
            action_val = self.q_table[x, y, :] # 4개의 액션에 대한 q값을 모두 가져옴
            action = np.argmax(action_val) # 10% 확률로 exploitation / 가장 큰 q값을 가지는 액션을 선택
        return action
    
    def update_table(self, history):
        # 한 에피소드에 해당하는 history를 입력으로 받아 q_table을 업데이트
        cum_reward = 0
        for transition in history[::-1]:
            s, a, r, s_prime = transition
            x, y = s
            # 몬테카를로 방식의 업데이트
            self.q_table[x, y, a] = self.q_table[x, y, a] + self.alpha * (cum_reward - self.q_table[x, y, a]) # q_table의 해당 위치의 q값을 업데이트
            cum_reward = self.gamma * cum_reward + r

    def anneal_eps(self):
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)

    def show_table(self):
        # 학습이 각 위치에서 어느 액션의 q값이 가장 높았는지 보여주는 함수
        q_lst = self.q_table.tolist()
        data = np.zeros((5, 7))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [32]:
# 몬테카를로 컨트롤

def main():
    env = GridWorld()
    agent = MCAgent()

    for n_epi in range(1000):
        done = False
        history = []

        s = env.reset()
        while not done:
            a = agent.select_action(s) # s = (0, 0), 랜덤하게 action 선택
            s_prime, r, done = env.step(a)
            history.append((s, a, r, s_prime))
            s = s_prime # state 업데이트
        agent.update_table(history) # 히스토리를 이용하여 에이전트를 업데이트
        agent.anneal_eps() # eps 0.1까지 decaying

    agent.show_table() # 학습이 끝난 결과를 출력


if __name__ == '__main__':
    main()

[[3. 2. 0. 3. 2. 2. 3.]
 [2. 3. 0. 2. 2. 2. 3.]
 [3. 3. 0. 1. 0. 2. 3.]
 [2. 3. 3. 1. 0. 3. 3.]
 [3. 2. 2. 1. 0. 2. 0.]]


In [33]:
# TD 컨트롤 1 - SARSA
class SAgent(MCAgent):
    def update_table(self, transition): # MC에서는 인자로 history를 받았지만, TD에서는 transition 하나만 받음
        s, a, r, s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime)
        self.q_table[x, y, a] = self.q_table[x, y, a] + 0.1 * (r + self.gamma * self.q_table[next_x, next_y, a_prime] - self.q_table[x, y, a])

In [34]:
def main():
    env = GridWorld()
    agent = SAgent()

    for n_epi in range(1000):
        done = False

        s = env.reset()
        while not done:
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s, a, r, s_prime))
            s = s_prime
        agent.anneal_eps()

    agent.show_table()

if __name__ == '__main__':
    main()

[[2. 3. 0. 2. 2. 2. 3.]
 [3. 3. 0. 2. 2. 2. 3.]
 [3. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [0. 1. 0. 1. 0. 2. 0.]]


In [35]:
# TD 컨트롤 2 - Q-Learning
class QAgent(MCAgent):
    def update_table(self, transition):
        s, a, r, s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        # Q-Learning 업데이트 식
        self.q_table[x, y, a] = self.q_table[x, y, a] + 0.1 * (r + np.amax(self.q_table[next_x, next_y, :]) - self.q_table[x, y, a]) # 벨만 최적 방정식을 사용

    def anneal_eps(self):
        self.eps -= 0.01 # Q-learning에서는 epsilon이 좀 더 천천히 줄어들도록 함
        self.eps = max(self.eps, 0.2)

In [36]:
def main():
    env = GridWorld()
    agent = QAgent()

    for n_epi in range(1000):
        done = False

        s = env.reset()
        while not done:
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s, a, r, s_prime))
            s = s_prime
        agent.anneal_eps()

    agent.show_table()

if __name__ == '__main__':
    main()

[[2. 3. 0. 2. 3. 2. 3.]
 [2. 3. 0. 2. 2. 2. 3.]
 [2. 3. 0. 1. 0. 2. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [2. 2. 1. 1. 0. 2. 0.]]


## 벨만 기대 방정식과 벨만 최적 방정식이 구현된 부분 다시 이해해보기!!